# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921688594266                   -0.69    5.9         
  2   -7.926165103831       -2.35       -1.22    1.0    331ms
  3   -7.926836734610       -3.17       -2.37    1.8    376ms
  4   -7.926861456975       -4.61       -3.01    2.5    400ms
  5   -7.926861629873       -6.76       -3.34    2.0    391ms
  6   -7.926861664802       -7.46       -3.69    1.8    325ms
  7   -7.926861680465       -7.81       -4.35    1.0    295ms
  8   -7.926861681755       -8.89       -4.81    2.1    347ms
  9   -7.926861681863       -9.97       -5.29    1.6    370ms
 10   -7.926861681872      -11.05       -5.90    1.9    341ms
 11   -7.926861681873      -12.00       -6.96    2.0    354ms
 12   -7.926861681873      -13.45       -7.36    3.1    413ms
 13   -7.926861681873   +    -Inf       -8.12    1.9    359ms


If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921672978851                   -0.69    5.9         
  2   -7.926161530949       -2.35       -1.22    1.0    327ms
  3   -7.926837401426       -3.17       -2.37    1.8    449ms
  4   -7.926861526332       -4.62       -3.04    2.8    391ms
  5   -7.926861643673       -6.93       -3.41    1.9    341ms
  6   -7.926861669717       -7.58       -3.79    1.8    328ms
  7   -7.926861680409       -7.97       -4.33    1.2    325ms
  8   -7.926861681763       -8.87       -4.83    2.0    403ms
  9   -7.926861681861      -10.01       -5.29    1.8    335ms
 10   -7.926861681872      -10.99       -5.96    1.9    347ms
 11   -7.926861681873      -12.00       -7.05    2.1    379ms
 12   -7.926861681873      -13.62       -7.45    2.9    427ms
 13   -7.926861681873      -15.05       -8.37    1.5    331ms


The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921682289845                   -0.69    5.6         
  2   -7.926171605532       -2.35       -1.22    1.0    324ms
  3   -7.926841039223       -3.17       -2.37    1.8    365ms
  4   -7.926864936909       -4.62       -3.03    2.8    468ms
  5   -7.926865048795       -6.95       -3.37    1.8    332ms
  6   -7.926865078219       -7.53       -3.75    1.8    320ms
  7   -7.926865091606       -7.87       -4.36    1.5    323ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")
